<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-T/blob/master/DataAlgo2020_T(014)_KnapSack_BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.3. Branch-and-Bound Search

On backtracking, solution candidates are reduced by checking the conditions the the solutions should fit.

On Branch-and-Bound approach, we step forward to reduce more candidates by predicting that the candidates won't meet the requirement.

To realize branch-and-bound approach, we need smart mind because we have to predict this time.

In this section, we pick up another very famous problem called Knapsack problem and go through brute-force, backtracking, and branch-and-bound.

**Reminder**  
On github, rendering might not be in good shape.  
To see the expected layout, open this page in Google Colaboratory.
To run one specific code cell in colab, click the icon on the left part or just type Ctrl + Enter.  


# Preparation

Connect the Jupyter environment and invoke a runtime. 
Mount your Google Drive by the procedure below.  
Change directory to the mounted point and make it as the working folder.  
By then, files are preserved even after you terminate the runtime environment.

In [ ]:
!echo "Mounting your Google Drive"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "Make a working folder and chnage directory to it"
%cd /content/drive/My\ Drive
%mkdir -p DataAlgo-T/014
%cd       DataAlgo-T/014
!ls

# Knapsack problem

**Problem definition**

Suppose you are a world-famous theif.

In the museum, there are N objects. Each object has its value and weight. 

At one night, you decide to pick up as many objects as possible from the museum with your knapsack. It can carry upto W kg, but if the total weight exceed the W kg, it will break and you will be arrested.

The problem here is, which objects should be picked up to maximize the total value you carry out?

(I ask you good students just imagine this situation, never do that!)

Note  all the values V<sub>i</sub> and weights W<sub>i</sub> (for i = 0, ..., N-1) are positive value.


**Heuristic search**

Smart people like you first think value per weight index for each object, sort objects by the index, and pick them up from the best one until the weight limit W comes.

It is very practical, and it gives the best performance if you can fractuate objects into any weights. However, on this problem, it might not give you the best solution. 



# Brute-force search for knapsack problem

AApparently a choice on one object is two-fold. You pick it up, or leave it. As there are N objects, the number of the total solution candidates is 2<sup>N</sup>.

Then, for each candidate, verify the candidate is within the weight limit. Among the successful solutions, find the largest value total and that is the answer.

2<sup>N</sup> is in the category of combinatorial explosion. So it will be very difficult to find the answer as N becomes large. By this property, it was once used to provide cipher in digital world.

In the program below, objects info is stored in the structured array obj[].

In [ ]:
%%writefile Knapsack-bf_E.c
// Knapsack problem solution by Bluteforce
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
void pickobject(int oid, knapsack_t ks_current){
	knapsack_t ks_next; // next status of the knapsack

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
    {

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next);

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
    {
    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Nothing to mark

	    // Leave object oid and proceed to the next object to examine
	    pickobject(oid + 1, ks_current);

        // Nothing to unmark
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Go
	pickobject(0, ks_init);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


Compile it and check no errors.

In [ ]:
!gcc -Wall -o Knapsack-bf_E Knapsack-bf_E.c

The first option is treated as weight limit. Run the program with the weight limit 112.

In [ ]:
!./Knapsack-bf_E 112

Count the displayed linesas one line corresponds to one candidate.
The last three lines are for the summary, so you can confirm the candidates are exactly 256. 
(wc -l counts the lins from stdin).

In [ ]:
!./Knapsack-bf_E 112 | wc -l

# Backtracking search for knapsack problem

Rewriting the Knapsack-bf_E.c to do backtracking should be rather easy as there is only one condition -- weight limit.

The updated program is shown below. Before you see the program, you are encouraged to discuss how it should be modified from the original Knapsack-bf_E.c program.

Make sure where is the backtracking part.






In [ ]:
%%writefile Knapsack-bt_E.c
// Knapsack problem solution by Backtrack
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
void pickobject(int oid, knapsack_t ks_current){
	knapsack_t ks_next; // next status of the knapsack

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
	if (ks_current.w + obj[oid].w <= wlimit) { // Weight limit condition

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next);

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
    {
    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Nothing to mark

	    // Leave object oid and proceed to the next object to examine
	    pickobject(oid + 1, ks_current);

        // Nothing to unmark
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Go
	pickobject(0, ks_init);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


You probably notice the two programs look very similar.
There is only one line for difference.
By using diff command (on linux), you can find it out.   
(No count on header part.)

In [ ]:
!diff Knapsack-bf_E.c Knapsack-bt_E.c

Compile it and check no errors.

In [ ]:
!gcc -Wall -o Knapsack-bt_E Knapsack-bt_E.c

Run it at the same condition.

In [ ]:
!./Knapsack-bt_E 112

And count the output lines too.

In [ ]:
!./Knapsack-bt_E 112 | wc -l

Uncounting the last 3 lines of summary, the candidates are now 174.  
About 1/3 of the candidates can be eliminated by the backtracking approach.  
This ratio would be affected by objects' property, processing order, and weight limit.



# Branch-and-Bound search for knapsack problem

On backtracking approach, we only can set the condition when we examine to pick up the object.  
Then, Can we do something even when we DO NOT pick up the object?  

--

When we decide not to pick up the object, we are getting a bit far from our goal -- maiximizing the total value -- because we miss the chance of adding the value of the object from now.

By promoting this idea, if "the current total value amount + the remaining values" to examine comes lower than the best record we have achieved in the past, we happan to realize that there is no chance of updating the best record beyond this point.

The "the current total value amount + the remaining values" is denoted by variable av (achievable value) in the program below.

The av is equal to the total sum of the museum at the beginning. Then, every time we decide not to pick up a object, its value is deduced from the av. If av (av_without_i in the program) comes lower than the tentative best achievement (max.v), there is no chance to update the best record beyond this point, so we need to go back.

The program below is a modifined version of the backtracking program. Read it very carefully and the new condition is set on "not-picking" choice this time.

On this program, the condition of backtracking is set to cull the candidates on "picking up" a object, and the condition of branch-and-bound is set to cull the conadidates on "not-picking up" a object.　 

In [ ]:
%%writefile Knapsack-bb_E.c
// Knapsack problem solution by Branch-and-Bound
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
// av         ... achievable value (at best) ... new for branch and bound
void pickobject(int oid, knapsack_t ks_current, int av){
	knapsack_t ks_next; // next status of the knapsack
	int av_without_i; // achievable value at best without (i)th object

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
	if (ks_current.w + obj[oid].w <= wlimit) { // Weight limit condition

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next, av); // av is same because you pick up this object.

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
	av_without_i = av - obj[oid].v;	

	// [Branch and bound] here!
	// If 'achievable value' is smaller than 'max.v', there is no hope to
	// get better solution beyond here. So, you need to proceed only when
	//  av_without_i > max.v
	if (av_without_i > max.v) {


    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Mark that no pick-up is made here : "stat[oid] = 0;" could be here.

	    // Leave object oid and proceed to the next object to examine
	  pickobject(oid + 1, ks_current, av_without_i); // less hope yet go

        // Unmark that recovery to the original state. "stat[oid] = 0;" could be here.
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};
	int totalvalue = 0;
	int i;

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Total Value
    for (i = 0; i < N; i++)
      totalvalue += obj[i].v;

    // Go
    pickobject(0, ks_init, totalvalue);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


See the difference (it is not easy to see this time by diff command).

In [ ]:
!diff Knapsack-bt_E.c Knapsack-bb_E.c

Compile it and check no errors.

In [ ]:
!gcc -Wall -o Knapsack-bb_E Knapsack-bb_E.c

Run it at the same condition.

In [ ]:
!./Knapsack-bb_E 112

Now the program can reach to generate the solution candidates only for twice.
Branch-and-Bound approach succeeded in culling the candidates drastically in this problem.

# Problems


1. Computation amount
Discuss the computation time on Knsapsack-bf_E.c, Knapsack-bt_E.c, and Knapsack-bb_E.c program with input size N (number of objects).


2. Measurement of the computation amount
Investigate the computation time on Knsapsack-bf_E.c, Knapsack-bt_E.c, and Knapsack-bb_E.c program. Enhance the problem to larger N (10 or more, you can specify values/weights by yourself for objects). Change the weight limit. On measuring the time, printf() should be omitted for fair comparison. Measurement should be the average of several runs. The table below is just an example. Find out good values to show the tendency.


|N|Weight Limit|bf[s]|bt[s]|bb[s]|
|-|---|--|--|--|
|8|100|???|???|???|
|8|120|???|???|???|
|8|140|???|???|???|
|16|100|???|???|???|
|16|120|???|???|???|
|16|140|???|???|???|








#**Course Info**

https://github.com/kameda-yoshinari/DataAlgo-T  
Course: Data structure and algorithm  
Department of Engineering Systems, University of Tsukuba,Japan.  
Author: KAMEDA, Yoshinari  
2020.05.19. -